In [16]:
import json
import csv
from pathlib import Path

1. Read each row in the CSV
1. Process the bounding box info
1. Crosswalk to an Aardvark object
1. Export JSON file

In [17]:
class aardvark(dict):
    def __init__(self, row_dict):
        # Hard Coded
        self.gbl_resourceClass_sm: list = ["Maps"] # Required
        self.dct_accessRights_s: str = "public"# Required
        self.gbl_mdModified_dt: str # datetime.now() formatted # Required
        self.gbl_mdVersion_s: str = "Aardvark" # Required
        self.schema_provider_s: str = "American Geographical Society Library – UWM Libraries"
        self.gbl_suppressed_b: bool = False

        # Variable
        self.dct_title_s: str = row_dict['short_title']
        self.dct_alternative_sm: list = [row_dict['full_title']]
        
        # Spatial Stuff
        self.locn_geometry = DMS_to_Envelope(row_dict['envelope'])
        self.dcat_bbox = self.locn_geometry

    def __str__(self):
        return f'{self.dct_title_s}: {self.dcat_bbox}'

    def toJSON(self):
        return json.dumps(vars(self)) # this is "more pythonic" than .__dict__
        



In [27]:
def DMS2DD(DMS_string):

    assert DMS_string.__class__ == str

    h = -1 if DMS_string[0] in ['W', 'S'] else 1
        
    DMS = {'d': DMS_string[1:4], 'm': DMS_string[4:6], 's': DMS_string[6:8]}

    #DD = d + (min/60) + (sec/3600) 
    DD = (float(DMS['d']) + (float(DMS['m'])/60) + (float(DMS['s'])/3600)) * h
    assert DD.__class__ == float
    return DD #returns DD as float

In [28]:
def DMS_to_Envelope(alma_string):
    DMS_list = alma_string.split(",")
    DMS_west = round(DMS2DD(DMS_list[0]), 6)
    DMS_east = round(DMS2DD(DMS_list[1]), 6)
    DMS_north = round(DMS2DD(DMS_list[2]), 6)
    DMS_south = round(DMS2DD(DMS_list[3]), 6)

    return json.dumps(f'ENVELOPE({DMS_west},{DMS_east},{DMS_north},{DMS_south})')


In [29]:
csv_path = Path(r'tables\Charts_merged.csv')
with open(csv_path) as csvfile:
    csvreader = csv.DictReader(csvfile)
    for row in csvreader:
        new_aardvark = aardvark(row)
        print(new_aardvark.toJSON())

{"gbl_resourceClass_sm": ["Maps"], "dct_accessRights_s": "public", "gbl_mdVersion_s": "Aardvark", "schema_provider_s": "American Geographical Society Library \u2013 UWM Libraries", "gbl_suppressed_b": false, "dct_title_s": "Buffalo Harbor, New York 1857", "dct_alternative_sm": ["Chart of Buffalo Harbor and head of Niagara River with the outlet of Lake Erie"], "locn_geometry": "\"ENVELOPE(-79.005278,-78.8325,42.971111,42.830556)\"", "dcat_bbox": "\"ENVELOPE(-79.005278,-78.8325,42.971111,42.830556)\""}
{"gbl_resourceClass_sm": ["Maps"], "dct_accessRights_s": "public", "gbl_mdVersion_s": "Aardvark", "schema_provider_s": "American Geographical Society Library \u2013 UWM Libraries", "gbl_suppressed_b": false, "dct_title_s": "Lake Survey Charts", "dct_alternative_sm": ["Chart of Saint Clair River"], "locn_geometry": "\"ENVELOPE(-82.633333,-82.4,42.663889,42.083333)\"", "dcat_bbox": "\"ENVELOPE(-82.633333,-82.4,42.663889,42.083333)\""}
{"gbl_resourceClass_sm": ["Maps"], "dct_accessRights_s": 